# Encapsulating custom code into Gym environments: A turning controller

**Summary:** In this tutorial, we will demonstrate how one can build controllers at different levels of abstraction by implementing Gym environments encoding variable amounts of preprogrammed computation. As an example, we will refactor the code we wrote in the [hybrid controller tutorial](https://neuromechfly.org/tutorials/hybrid_controller.html#building-a-hybrid-controller) and implement turning by modulating the amplitude and frequency of CPGs asymmetrically.

## Gym environments and MDP

So far, we have interacted with the `NeuroMechFly` class which implements the [Gym interface](https://gymnasium.farama.org/): it has an action space (input given by the user), an observation space (output returned to the user), a `.step(...)` method, a `.reset(...)` method, and a `.render(...)` method. The action and observation spaces of `NeuroMechFly` are the following:

- Action space:
    - "joints": 42-dimensional real vector ∈ [0, 2π] indicating joint angles (in default position control mode)
    - "adhesion": 6-dimensional integer vector ∈ {0, 1} indicating whether adhesion is turned on for each leg
- Observation space:
    - "joints": shape (3, 42) real array indicating joint angles (0th row), velocities (1st row), and torques (2nd row)
    - "fly": shape (4, 3) real array indicating fly xyz positions (0th row), xyz velocities (1st row), yaw-pitch-roll angles (2nd row), and yaw-pitch-roll velocities (3rd row)
    - "contact_forces": shape (N, 3) - contact forces, in xyz directions, of the N segments defined in `contact_sensor_placements`
    - "end_effectors": shape (6, 3) - xyz positions of the six legs in the order of LF, LM LH, RF, RM, RH
    - "fly_orientation": shape (3,) - unit vector indicating the fly's heading orientation


The FlyGym package is designed to be expandable: the user can implement their own Gym environments with different action and observation spaces and implement different logics (eg. preprogrammed premotor computation and sensory processing). This is demonstrated in the figure below:

<img src="https://github.com/NeLy-EPFL/_media/blob/main/flygym/mdp.png?raw=true" alt="rule_based" width="500"/>

In the hybrid controller that [we have implemented](https://neuromechfly.org/tutorials/hybrid_controller.html#building-a-hybrid-controller), the underlying CPG network and the correction mechanisms can be considered the user-defined premotor computation (purple). The whole controller can be considered the box indicating the Markov Decision Process (MDP). Here, we will add a 2D descending representation encoding turning so that the action and observation spaces of our MDP "box" are the following:

- Action space: a 2-dimensional real vector ∈ [-1, 1] describing the velocity on each side
- Observation space: same as above (no sensory processing logic indicated in cyan)

## Approach for turning

We will use a 2-dimensional descending representation $[\delta_L, \delta_R] \in [-1, 1]^2$ to modulate the amplitude and direction of the CPGs on each side. Precisely, we will modulate the intrinsic amplitude $R'$ and intrinsic frequency $\nu'$ on each side by:

$$
R'(\delta) = |\delta|
$$

$$
\nu_i'(\delta) = \begin{cases}
\nu_i   & \text{if } \delta>0\\
-\nu_i  & \text{otherwise}
\end{cases}
$$

In other words, the maginuted of the descending signal controls the amplitude of stepping; the sign of the descending signal controls the direction of stepping. Of course, this is a very simplified model of turning. Perhaps the most unrealisitc aspect of it is that it assumes the step size to linearly span from 0 to 1x the recorded "normal" step size. This is something that the user can improve.

## Implementing the `HybridTurningController` class